In [ ]:
# Install any missing packages
!pip install networkx --quiet

import pandas as pd
import numpy as np
import networkx as nx
import matplotlib.pyplot as plt

%matplotlib inline


In [ ]:
from google.colab import files
import io

print("Upload your BTS T-100 CSV file...")
uploaded = files.upload()

# Take the first uploaded file
filename = list(uploaded.keys())[0]
print(f"Using file: {filename}")

# Read CSV into DataFrame
df = pd.read_csv(io.BytesIO(uploaded[filename]))

print("Columns in the dataset:")
print(df.columns)

# Peek at the data
df.head()


Upload your BTS T-100 CSV file...


IndexError: list index out of range

In [ ]:
## 1. Build the Yearly Airport Network

- Nodes: all unique airports that appear in `ORIGIN` or `DEST`
- Edges: directed from `ORIGIN` → `DEST`
- Edge weight: number of flights (rows) on that route across the whole year


In [ ]:
# Ensure ORIGIN and DEST exist
assert "ORIGIN" in df.columns and "DEST" in df.columns, "Columns ORIGIN/DEST not found!"

# 1. Nodes: all unique airports
airports = set(df["ORIGIN"]).union(set(df["DEST"]))
print(f"Number of unique airports (nodes): {len(airports)}")

# 2. Edges: group by (ORIGIN, DEST) over all rows/months
edge_df = (
    df.groupby(["ORIGIN", "DEST"], as_index=False)
      .size()
      .rename(columns={"size": "weight"})
)

print("Sample of aggregated edge list:")
edge_df.head()


In [ ]:
# Build directed weighted graph G = (V, E, W)
G = nx.DiGraph()
G.add_nodes_from(airports)

for _, row in edge_df.iterrows():
    origin = row["ORIGIN"]
    dest = row["DEST"]
    weight = row["weight"]
    if origin != dest:  # optional: drop self-loops
        G.add_edge(origin, dest, weight=weight)

print(f"|V| (nodes): {G.number_of_nodes()}")
print(f"|E| (edges): {G.number_of_edges()}")


## 2. Network-Level Statistics

We will compute:

- |V|: number of airports
- |E|: number of directed routes
- Density ρ = |E| / (|V| (|V| − 1))
- Degree and strength (weighted degree) distributions
- Largest weakly connected component, average path length, diameter
- Clustering coefficient
- Simple small-world coefficient (compare to random graph)


In [ ]:
# Basic stats
V = G.number_of_nodes()
E = G.number_of_edges()
density = nx.density(G)

print(f"|V| (nodes): {V}")
print(f"|E| (edges): {E}")
print(f"Density ρ = {density:.6f}")

# In-/out-degree and total degree
in_deg = dict(G.in_degree())
out_deg = dict(G.out_degree())
total_deg = {n: in_deg.get(n, 0) + out_deg.get(n, 0) for n in G.nodes()}

# Weighted degrees (strength)
strength_in = dict(G.in_degree(weight="weight"))
strength_out = dict(G.out_degree(weight="weight"))
strength_total = {
    n: strength_in.get(n, 0) + strength_out.get(n, 0)
    for n in G.nodes()
}

print("Example (first 5 airports) - total degree:")
list(total_deg.items())[:5]


In [ ]:
# Work on the largest weakly connected component for global metrics
if nx.is_weakly_connected(G):
    G_lcc = G
else:
    largest_cc_nodes = max(nx.weakly_connected_components(G), key=len)
    G_lcc = G.subgraph(largest_cc_nodes).copy()

print(f"Largest weakly connected component has {G_lcc.number_of_nodes()} nodes.")

# Convert to undirected for path length & clustering
G_lcc_und = G_lcc.to_undirected()

# Average shortest path length and diameter
avg_path_length = nx.average_shortest_path_length(G_lcc_und)
diameter = nx.diameter(G_lcc_und)

print(f"Average shortest path length (L): {avg_path_length:.3f}")
print(f"Diameter: {diameter}")

# Clustering coefficient
C = nx.average_clustering(G_lcc_und)
print(f"Average clustering coefficient (C): {C:.4f}")



In [ ]:
import random

# Degree sequence of LCC
deg_seq = [deg for _, deg in G_lcc_und.degree()]

# Fix seeds for reproducibility
random.seed(0)
np.random.seed(0)

# Configuration model random graph with same degree sequence
rand_graph = nx.configuration_model(deg_seq)
rand_graph = nx.Graph(rand_graph)  # simple undirected
rand_graph.remove_edges_from(nx.selfloop_edges(rand_graph))

# Largest component of random graph
largest_rand_nodes = max(nx.connected_components(rand_graph), key=len)
rand_lcc = rand_graph.subgraph(largest_rand_nodes).copy()

C_rand = nx.average_clustering(rand_lcc)
L_rand = nx.average_shortest_path_length(rand_lcc)

small_world_sigma = (C / C_rand) / (avg_path_length / L_rand)

print(f"Random graph C_rand: {C_rand:.4f}")
print(f"Random graph L_rand: {L_rand:.3f}")
print(f"Small-world coefficient σ = (C/C_rand) / (L/L_rand) = {small_world_sigma:.3f}")


## 3. Degree and Strength Distributions (CCDF)

We will plot:

- CCDF of total degree (unweighted)
- CCDF of total strength (weighted degree = total flights)
on log–log axes to highlight heavy tails.


In [ ]:
def plot_ccdf(values_dict, xlabel, title):
    """
    Plot CCDF (P(X > x)) on log-log axes for a dict of node -> value.
    """
    data = np.array(list(values_dict.values()))
    data = data[data > 0]  # remove zeros to avoid log(0)

    if len(data) == 0:
        print(f"No positive values for {title}, skipping plot.")
        return

    sorted_data = np.sort(data)
    n = len(sorted_data)
    ccdf = 1.0 - np.arange(1, n + 1) / n

    plt.figure()
    plt.loglog(sorted_data, ccdf, marker=".", linestyle="none")
    plt.xlabel(xlabel)
    plt.ylabel("CCDF")
    plt.title(title)
    plt.grid(True, which="both")
    plt.tight_layout()
    plt.show()

# Degree CCDF
plot_ccdf(total_deg, "Degree", "Degree distribution (CCDF)")

# Strength CCDF
plot_ccdf(strength_total, "Weighted degree (strength)", "Strength distribution (CCDF)")


## 4. Traffic Concentration in Top-k Airports

We measure:

- Total flights handled by each airport = total strength
- Fraction of all flights carried by the top k airports (k = 5, 10, 20)
to show hub dominance.


In [ ]:
# Sort airports by total strength (sum of in+out weights)
strength_sorted = sorted(strength_total.values(), reverse=True)
total_strength_val = sum(strength_sorted)

ks = [5, 10, 20]
fractions = []

for k in ks:
    k_eff = min(k, len(strength_sorted))
    frac = sum(strength_sorted[:k_eff]) / total_strength_val
    fractions.append(frac)
    print(f"Top {k_eff} airports carry {frac*100:.2f}% of total flights")

plt.figure()
plt.plot(ks, fractions, marker="o")
plt.xlabel("k (top-k airports by traffic)")
plt.ylabel("Fraction of total traffic")
plt.title("Traffic concentration in top-k airports")
plt.grid(True)
plt.tight_layout()
plt.show()


## 5. Top Hubs (for Section 5.1 text)

We list top airports by:
- Total traffic (weighted degree / strength)
- Total degree (number of distinct routes)
You can copy these names & numbers into your report.


In [ ]:
# Top 10 by strength (total flights)
top10_strength = sorted(strength_total.items(), key=lambda x: x[1], reverse=True)[:10]
print("Top 10 airports by total traffic (strength):")
for airport, val in top10_strength:
    print(f"{airport}: {val} flights (year total)")

# Top 10 by total degree
top10_degree = sorted(total_deg.items(), key=lambda x: x[1], reverse=True)[:10]
print("\nTop 10 airports by total degree (distinct routes):")
for airport, val in top10_degree:
    print(f"{airport}: degree = {val}")
